In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
vp_week = pd.read_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx") #ventas perdidas
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios

In [2]:
df.head()

,origin,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name
0,False,59000.0,1.0,[BCE00609125] G609 FILTRO COMBUSTIBLE MERCEDES...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
1,False,105000.0,1.0,[DAE09118025] DA9118 FILTRO AIRE SEGURIDAD USE...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
2,False,52500.0,1.0,[BLE00668125] G668 FILTRO ACEITE FREIGTHLINER ...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
3,False,160300.0,1.0,[DAB02970025] DA2970 FILTRO AIRE BUS SKANIA MO...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
4,BD1\OUT\60234,710400.0,12.0,[BLS00459125] GS459 FILTRO ACEITE JCB 320/0413...,FCOT26332,2025-09-23 14:41:35,PRINCIPAL COTA


# EDA

## Facturas

In [3]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])


df = df.drop(columns=["origin"])

df_externos_bogota = df[(df["store_name"]=="PRINCIPAL COTA") & (df["Sucursal_tipo"]=="VENDEDOR EXTERNO")]

df = df.drop(df_externos_bogota.index)


In [4]:
df.groupby(["store_name","Bodega","Sucursal_tipo"])["quantity"].sum()

store_name             Bodega                             Sucursal_tipo   
PRINCIPAL COTA         PRINCIPAL COTA                     MOSTRADOR           15897.0
SUCURSAL BARRANQUILLA  BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO      113.0
                       SUCURSAL BARRANQUILLA              MOSTRADOR           23855.0
                                                          VENDEDOR EXTERNO    74265.0
                       SUCURSAL CALI                      VENDEDOR EXTERNO        2.0
SUCURSAL BUCARAMANGA    SUCURSAL BUCARAGAMNGA             VENDEDOR EXTERNO    46238.0
                       PRINCIPAL COTA                     VENDEDOR EXTERNO        2.0
                       SUCURSAL BUCARAMANGA               MOSTRADOR           18386.0
SUCURSAL CALI          BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO     1966.0
                       EXP.REP.DOMINICANA                 VENDEDOR EXTERNO     1961.0
                       PRINCIPAL COTA                     VENDEDO

In [5]:
df[df["product_name"].str.contains("DCE00833189")].sort_values(by=["quantity"],ascending=False)

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
96972,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FBAQ13002,2025-04-02 21:04:23,SUCURSAL BARRANQUILLA,VENDEDOR EXTERNO,BD6,SUCURSAL BARRANQUILLA
7393,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43208 Mostrador Calle 6 T2/21164,2025-09-10 17:49:58,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
187962,514800.0,36.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV1E8545 Mostrador Cota T2/1936,2024-10-15 20:33:42,PRINCIPAL COTA,MOSTRADOR,<NA>,PRINCIPAL COTA
115741,540000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E37805 Mostrador Calle 6 T3/21299,2025-02-28 14:32:09,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
53469,477000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E40849 Mostrador Calle 6 T3/22423,2025-06-20 14:02:51,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
...,...,...,...,...,...,...,...,...,...
40238,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E41515 Mostrador Calle 6 T3/22769,2025-07-15 20:23:07,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
31240,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV9E1464 Mostrador Cali T1/1842,2025-07-30 18:40:36,SUCURSAL CALI,MOSTRADOR,<NA>,SUCURSAL CALI
113646,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FMDE1990,2025-03-05 14:52:01,SUCURSAL MEDELLIN,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN
113655,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FMDE1989,2025-03-05 14:50:20,SUCURSAL MEDELLIN,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN


In [6]:
df_externos_bogota = df[(df["store_name"]=="SUCURSAL CALI") & (df["Bodega"]=="EXP.REP.DOMINICANA")]

df_externos_bogota

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
56301,765600.0,12.0,[DAB02805025] DA2805 FILTRO AIRE 1_ PERKINS PL...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56302,571200.0,12.0,[DAR02005025] DA2005 FILTRO AIRE FOTON dob cab...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56303,56800.0,2.0,[DAB04864025] DA4864 FILTRO AIRE 2_ CATERPILLA...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56304,401200.0,4.0,"[DAB08027025] DA8027 FILTRO AIRE ATLAS COPCO, ...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56305,131900.0,1.0,[DAR02997025] DA2997 FILTRO AIRE IVECO 4127221...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
...,...,...,...,...,...,...,...,...,...
56373,2862500.0,125.0,[DAB02612025] DA2612 FILTRO AIRE 1_ TOYOTA MON...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56374,2062500.0,125.0,[DAB14772025] DA4772A FILTRO AIRE INTERNO BOBC...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56375,2062500.0,125.0,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56376,7468750.0,125.0,[DAB02690025] DA2690 FILTRO AIRE- FREIGTHLINER...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA


# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [7]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)
    .agg(
        sales=("quantity", "sum"),              # suma de unidades vendidas
        num_facturas=("date_invoice", "nunique")  # cantidad de facturas distintas
    )
)


# Resultado
sales_by_week.sample(10)



,store_name,product_ref,Semana,ano,mes,dia,sales,num_facturas
5569,PRINCIPAL COTA,DCS00679189,2024-10-14,2024,10,14,4.0,1
55087,SUCURSAL CALLE 6,BLS10018125,2025-07-21,2025,7,21,5.0,2
35612,SUCURSAL BUCARAMANGA,DAR03113025,2025-05-19,2025,5,19,2.0,1
85508,SUCURSAL NORTE,DAB08131025,2024-10-21,2024,10,21,1.0,1
78183,SUCURSAL MEDELLIN,DCE00046137,2024-11-04,2024,11,4,2.0,2
31323,SUCURSAL BUCARAMANGA,DAB02932025,2024-11-25,2024,11,25,6.0,2
77118,SUCURSAL MEDELLIN,DAR02270025,2024-12-16,2024,12,16,1.0,1
34472,SUCURSAL BUCARAMANGA,DAE08142025,2025-02-17,2025,2,17,2.0,1
27044,SUCURSAL BUCARAMANGA,BCS00462125,2025-06-23,2025,6,23,1.0,1
3645,PRINCIPAL COTA,DAB04872025,2024-11-11,2024,11,11,1.0,1


In [8]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales","num_facturas"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","veces_vp"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [9]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day


df_merged.head(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
0,PRINCIPAL COTA,AC10388020,2025-09-22,0.0,0.0,30.0,2.0,2025,9,22
1,PRINCIPAL COTA,AGB0GRAS030,2025-07-28,0.0,0.0,1.0,1.0,2025,7,28
2,PRINCIPAL COTA,AHB0TO30132,2025-01-13,0.0,0.0,1.0,1.0,2025,1,13
3,PRINCIPAL COTA,AHB80W90030,2025-07-07,0.0,0.0,1.0,1.0,2025,7,7
4,PRINCIPAL COTA,AHB80W90050,2025-08-11,0.0,0.0,1.0,1.0,2025,8,11
5,PRINCIPAL COTA,AHBNTO68030,2024-10-21,1.0,1.0,0.0,0.0,2024,10,21
6,PRINCIPAL COTA,AHBNTO68030,2025-01-27,1.0,1.0,1.0,1.0,2025,1,27
7,PRINCIPAL COTA,AHBNTO68030,2025-02-03,0.0,0.0,1.0,1.0,2025,2,3
8,PRINCIPAL COTA,AHBNTO68030,2025-02-10,0.0,0.0,1.0,1.0,2025,2,10
9,PRINCIPAL COTA,AHBNTO68030,2025-02-17,0.0,0.0,4.0,1.0,2025,2,17


In [10]:
df_merged.query("product_ref == 'BAE05001125'")

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
15286,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-20,2.0,1.0,0.0,0.0,2025,1,20
15287,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
15288,SUCURSAL BARRANQUILLA,BAE05001125,2025-03-24,0.0,0.0,3.0,3.0,2025,3,24
15289,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-08,1.0,1.0,0.0,0.0,2025,9,8
92499,SUCURSAL MEDELLIN,BAE05001125,2025-02-17,0.0,0.0,1.0,1.0,2025,2,17
92500,SUCURSAL MEDELLIN,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
92501,SUCURSAL MEDELLIN,BAE05001125,2025-06-09,0.0,0.0,1.0,1.0,2025,6,9
92502,SUCURSAL MEDELLIN,BAE05001125,2025-08-18,0.0,0.0,1.0,1.0,2025,8,18
112066,SUCURSAL NORTE,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24


In [11]:
df_merged["product_ref"].nunique()

4309

## NORMALIZAR DF DE COSTOS

In [12]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p["product_ref"].nunique()

4835

## EMA 

In [13]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # peso de EMA
                           n_init_weeks=12,   # semanas para inicializar EMA
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")
    df[week_col] = df[week_col].dt.to_period('W-MON').dt.start_time

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # ---------- Crear combinaciones completas ----------
    stores   = df["store_name"].unique()
    products = df["product_ref"].unique()


    # Crear rango completo desde min hasta max semana
    weeks = pd.period_range(df[week_col].min(), df[week_col].max(), freq="W-MON").to_timestamp()


    full_index = pd.MultiIndex.from_product([stores, products, weeks],
                                            names=["store_name", "product_ref", week_col])

    df = df.set_index(["store_name", "product_ref", week_col]).reindex(full_index).reset_index()

    # Rellenar ventas y perdidas con 0 en combinaciones faltantes
    df[sales_col] = df[sales_col].fillna(0)
    df[lost_col]  = df[lost_col].fillna(0)

    # Ordenar
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    # ---------- Calcular demanda ajustada y EMA ----------
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # Inicialización EMA: promedio de primeras n_init_weeks ventas
        init_n = min(n_init_weeks, L)
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1
            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2
                demand = s + 0.5 * ema_prev

            # Guardar redondeando
            demanda[i] = round(demand, 6)

            # Calcular EMA
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 6)
            ema_arr[i] = ema

            # actualizar para siguiente
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

        

    # Concat resultados
    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
df_with_demand1 = compute_demand_and_ema(df_merged, alpha=0.2, n_init_weeks=12) #estaba en 16

#APROX3min

In [14]:
df_with_demand1.groupby("store_name")["product_ref"].nunique()

store_name
PRINCIPAL COTA           4309
SUCURSAL BARRANQUILLA    4309
SUCURSAL BUCARAMANGA     4309
SUCURSAL CALI            4309
SUCURSAL CALLE 6         4309
SUCURSAL MEDELLIN        4309
SUCURSAL NORTE           4309
SUCURSAL VALLADOLID      4309
Name: product_ref, dtype: int64

In [15]:
def add_deviation_and_classification(df, ema_col="EMA", sales_col="sales", week_col="Semana", n_weeks=12): ##PERILLA SEMANAS 
    df = df.copy()
    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        
        sales = g[sales_col].to_numpy(dtype=float)
        ema   = g[ema_col].to_numpy(dtype=float)
        L = len(sales)

        # Inicializamos columnas
        g["desviacion_ema%"] = 0.0
        g["coef_ventas"] = 0.0

        if L < 2:
            out_groups.append(g)
            continue

        for i in range(L):
            # Usar solo hasta la semana actual (i)
            window_start = max(0, i - n_weeks)       # ventana de 12 semanas
            window_sales = sales[window_start:i]     # semanas previas
            window_ema   = ema[window_start:i]

            if len(window_sales) < 2:
                continue

            promedio_prev = np.mean(window_sales)
            if promedio_prev == 0:
                promedio_prev = 1e-6

            # --- Desviación usando EMAs previos ---
            desv = np.std(window_ema, ddof=1) / promedio_prev if promedio_prev > 0 else 0

            # --- Coeficiente de ventas ---
            coef = (max(window_sales) - min(window_sales)) / promedio_prev if promedio_prev > 0 else 0

            g.at[i, "desviacion_ema%"] = round(desv,2)
            g.at[i, "coef_ventas"] = coef

        out_groups.append(g)

    return pd.concat(out_groups, ignore_index=True)





df_with_demand2 = add_deviation_and_classification(df_with_demand1)
df_with_demand2 = df_with_demand2[["store_name","product_ref","Semana","num_facturas","sales","lost_sales","demanda_ajustada","EMA","desviacion_ema%","coef_ventas"]]

# Demora mas o menos 8 min

In [16]:
df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)

,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas
252727,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0
252756,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0
252726,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0
252755,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0
252725,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0
252754,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0
252724,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0
252723,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0
252722,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0
252721,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0


In [17]:
df_with_demand2["semana_num"] = df_with_demand2["Semana"].dt.isocalendar().week
df_with_demand2["año"] = df_with_demand2["Semana"].dt.year


### AGREGAR SEMANA CSV

In [18]:
df_demand_2025 = df_with_demand2[df_with_demand2["año"]==2025] #Ajustar año

demand_2025 = df_demand_2025[
    ["store_name","product_ref","año","Semana","semana_num",
     "num_facturas","EMA","demanda_ajustada","desviacion_ema%","coef_ventas","lost_sales"]
]


demand_2025_37 = demand_2025[demand_2025["semana_num"]==37]  #Ajustar semana numero

df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)




,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas,semana_num,año
252727,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0,9,2025
252756,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0,38,2025
252726,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0,8,2025
252755,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0,37,2025
252725,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0,7,2025
252754,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0,36,2025
252724,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0,6,2025
252723,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0,5,2025
252722,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0,4,2025
252721,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0,3,2025


## UNIR COSTO

In [19]:
df_merge_def = pd.merge(
    demand_2025_37,
    df_p_unique,
    on="product_ref",
    how="outer" #para incluir todos , estaba antes left
).fillna(0)

df_merge_def.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202537.csv")

#merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios

merge_def =  df_merge_def.copy()
merge_def.groupby("store_name")["product_ref"].nunique()

store_name
0                         566
PRINCIPAL COTA           4309
SUCURSAL BARRANQUILLA    4309
SUCURSAL BUCARAMANGA     4309
SUCURSAL CALI            4309
SUCURSAL CALLE 6         4309
SUCURSAL MEDELLIN        4309
SUCURSAL NORTE           4309
SUCURSAL VALLADOLID      4309
Name: product_ref, dtype: int64

In [20]:
merge_def[merge_def["store_name"]==0].sample(10)

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
15113,0,CS007658222,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,7751.50
4917,0,BAR04986125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
21510,0,DAE02483025,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9000.01
23221,0,DAE05566010,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2054.02
22866,0,DAE05162010,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3936.34
10231,0,BHE00873125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,27323.55
23170,0,DAE05501024,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,8400.00
1736,0,BAB09247125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,113147.00
33007,0,DHE00673111,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
9874,0,BHE00741125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,18767.51


In [21]:
merge_def.query("product_ref == 'BAE05001125'")

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
3730,PRINCIPAL COTA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3731,SUCURSAL BARRANQUILLA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.160257,0.0,0.69,12.0,0.0,90836.27
3732,SUCURSAL BUCARAMANGA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3733,SUCURSAL CALI,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3734,SUCURSAL CALLE 6,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3735,SUCURSAL MEDELLIN,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3736,SUCURSAL NORTE,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3737,SUCURSAL VALLADOLID,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
